## Test Mel Spectogram

### Import library

In [ ]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import os
from mutagen.mp3 import MP3

### Spectogram

In [ ]:
def mp3_to_melspec(mp3_file, output_path, *, sr=22050, duration=30, n_fft=2048, n_mels=244, hop_length=2713):

    y, sr = librosa.load(mp3_file, sr=sr, duration=duration)

    # calculete mel-spectogram
    mel_spec = librosa.feature.melspectrogram(
    y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels
    )
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    normalized = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db)-np.min(mel_spec_db))  #

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    librosa.display.specshow(normalized, sr=sr, hop_length=hop_length)
    plt.savefig(str(output_path), bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
AUDIO_PATH = Path.cwd() / 'project_data' / 'audio_data'
SPECT_PATH = Path.cwd() / 'project_data' / 'spectrogram_data'

In [ ]:
def is_longer_than_25_seconds(file_path):
    for mp3_file in sorted(file_path.rglob("*.mp3")):
        try:
            audio = MP3(mp3_file)
            duration = audio.info.length  # długość w sekundach (float)
            if duration < 25:
                print(mp3_file)
        except Exception as e:
            print(f"Błąd podczas przetwarzania pliku: {e}")
            return False


is_longer_than_25_seconds(AUDIO_PATH)

In [ ]:
start_from = "093521.mp3"
skip = True
for mp3_file in sorted(AUDIO_PATH.rglob("*.mp3")):
    if skip:
        if mp3_file.name == start_from:
            skip = False
        else:
            continue
    try:
        file_stem = mp3_file.stem
        output_file = SPECT_PATH / f"{file_stem}.png"
        mp3_to_melspec(mp3_file, output_file)
    except Exception as e:
        print(f"Błąd przy pliku {mp3_file}: {e}")



In [ ]:
df = pd.read_csv('project_data/tracks.csv')

df['filename'] = df['track_id'].apply(lambda x: f'{x:06d}.png')
new_data = df[['filename', 'genre']]
new_data.to_csv('dataset/spectrogram.csv', index=False)